In [1]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import pandas as pd 
import inspect
import sys, logging

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)




In [2]:
def eval_scm(scm, nsamples, **kwargs):
    values = dict(N = nsamples)
    values.update(kwargs)
    
    for varname, func in scm.items():
        argnames = list(inspect.signature(func).parameters.keys())
        logging.info(f"{varname} : f({argnames})")
        args = { name : values[name] for name in argnames }
        values[varname] =func(**args)
    return values

def reeval_scm_with_do(scm, base, **do):    
    values = {}
    values.update(base)    
    modified = set()
    for varname, func in scm.items():
        if varname in do:
            logging.info(f"DO {varname}")
            func = do[varname]                            

        argnames = list(inspect.signature(func).parameters.keys())                        
        if varname not in do and not modified.intersection(argnames):
            continue
        modified.add(varname)
        args = { name : values[name] for name in argnames }
        logging.info(f"{varname} : f({argnames})")
        values[varname] =func(**args)
    return values


def ground_truth_ate(scm, values, treatment, response):
    logging.info("Calculating ground truth")
    values_0 =  reeval_scm_with_do(SCM, values, **{treatment : lambda N: torch.zeros(N)})
    values_1 =  reeval_scm_with_do(SCM, values, **{treatment : lambda N: torch.ones(N)})
    return (values_1[response] - values_0[response]).mean()


In [3]:
class CondMeanDiff:

    def __init__(self, treatment, response):
        self.treatment_name = treatment
        self.response_name = response

    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        return response[treatment >= 0.5].mean() - response[treatment < 0.5].mean()    


class ATEStratified:
    def __init__(self, treatment, response, strat_func):
        self.treatment_name = treatment
        self.response_name = response
        self.strat_func = strat_func        

    def __call__(self, values):        
        response = values[self.response_name]
        treatment = values[self.treatment_name]

        keys = self.strat_func(values)
        unique_keys =set(keys.tolist())
        ate = 0.0
        total_weight = 0
        for k in unique_keys:        
            is_treatment = (treatment > 0.5)
            treatment_mask = (keys == k) & is_treatment
            control_mask = (keys == k) & (~is_treatment)
            if (treatment_mask.sum() == 0 or control_mask.sum() == 0):
                logging.info(f"skipping group {k}")
                continue
            group_ate = (response[treatment_mask].mean() - response[control_mask].mean())
            group_weight = ((keys == k)*1).sum()
            total_weight += group_weight
            ate = ate + group_ate*group_weight
        ate = ate / total_weight
        return ate


class SplitStratify:
    def __init__(self, varname, splits):
        self.varname = varname
        self.splits = torch.Tensor(splits)

    def __call__(self, values):
        vals = values[self.varname]
        keys = torch.zeros(vals.shape)
        for bar in self.splits:
            keys += (vals < bar )
        return keys

class QuantStratify:
    def __init__(self, varname, ngroups):
        self.varname = varname
        self.ngroups = ngroups

    def __call__(self, values):
        vals = values[self.varname]
        keys = torch.zeros(vals.shape)
        quantiles = torch.quantile(vals, torch.arange(self.ngroups)[1:]/float(self.ngroups))
        for q in quantiles:
            keys += (vals <  q)
        return keys

def enrich_propensity(values, target_name, covariate_names, propensity_name="propensity", model=None):
    # Convert target to numpy
    y = values[target_name].cpu().numpy()
    if y.ndim > 1 and y.shape[1] == 1:
        y = y.flatten()

    # Concatenate covariates (converted to numpy)
    X = torch.stack([values[name] for name in covariate_names]).numpy().transpose()    
    # Create and fit sklearn logistic regression
    if model is None:
        model = LogisticRegression(max_iter=1000)
    logging.info(f"fitting model for propensity: '{propensity_name}'")
    model.fit(X, y)

    # Predict probabilities (for positive class)
    probs = model.predict_proba(X)[:, 1]

    # Convert back to torch tensor
    probs_tensor = torch.from_numpy(probs).float()
    values[propensity_name] = probs_tensor

class ATEPropensity:

    def __init__(self, treatment, response, propensity):
        self.treatment_name = treatment
        self.response_name = response
        self.propensity_name = propensity

    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        prop = values[self.propensity_name]
        
        treatment_mask = (treatment >= 0.5)
        
        return (
            (response[treatment_mask]/prop[treatment_mask]).sum() / (1.0/prop[treatment_mask]).sum() -
            (response[~treatment_mask]/(1-prop[~treatment_mask])).sum() / (1.0/(1-prop[~treatment_mask])).sum()
        )

class ATEImpute:

    def __init__(self, treatment, response, conditions, mgen=None):
        self.treatment_name = treatment
        self.response_name = response
        self.condition_names = conditions
        self.mgen = mgen or (lambda: GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3))
        
    def __call__(self, values):
        response = values[self.response_name]
        treatment = values[self.treatment_name] 
        
        y = response.numpy()

        # Concatenate covariates (converted to numpy)
        X = torch.stack([values[name] for name in [self.treatment_name] + self.condition_names]).numpy().transpose()    
        model = self.mgen()
        logging.info(f"fitting model for imputation")
        model.fit(X, y)

        treatment_mask = (treatment > 0.5)
        treatment_counterfactual_covs = X[(treatment_mask.numpy()), :]        
        treatment_counterfactual_covs[:,0] = 0
        
        treatment_counterfactual_resp = torch.from_numpy(model.predict(treatment_counterfactual_covs)).float()        

        control_mask = ~treatment_mask
        control_counterfactual_covs = X[(control_mask.numpy()), :]
        control_counterfactual_covs[:,0] = 1
        control_counterfactual_resp = torch.from_numpy(model.predict(control_counterfactual_covs)).float()        

        ate = (
            (response[treatment_mask] -  treatment_counterfactual_resp).sum() + 
            (control_counterfactual_resp - response[control_mask]).sum()) / response.numel()
        return ate
        

def apply_filter(values, mask):
    return {k : (v[mask] if type(v) == torch.Tensor else v) for k,v in values.items() }

In [4]:
def show(**kwargs):
    return pd.DataFrame({k:v.unsqueeze(0) for k,v in kwargs.items()})

## Sanity - No causality
(Two items of the same gerne)

In [11]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.2 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.2 + 0.5*U))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))



2025-07-21 15:46:02,023 - INFO - U : f(['N'])
2025-07-21 15:46:02,027 - INFO - T : f(['N', 'U'])
2025-07-21 15:46:02,029 - INFO - Y : f(['N', 'U', 'T'])
2025-07-21 15:46:02,032 - INFO - fitting model for propensity: 'propensity'
2025-07-21 15:46:02,137 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.250818,-0.00048,-0.00042,-0.000477


## Causality with Confounder

In [6]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.5*U + 0.2*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

# GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-21 15:43:10,048 - INFO - U : f(['N'])
2025-07-21 15:43:10,052 - INFO - T : f(['N', 'U'])
2025-07-21 15:43:10,055 - INFO - Y : f(['N', 'U', 'T'])
2025-07-21 15:43:10,059 - INFO - fitting model for propensity: 'propensity'
2025-07-21 15:43:10,169 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.472555,0.198604,0.198704,0.198605


## Causality with a weak Confounder
(For example Godfather-I, Godfather II)

In [7]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.5*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.05 + 0.05*U  + 0.5*T))*1.0
}
values = eval_scm(SCM, 100000)
enrich_propensity(values, "T", ["U"])#, model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))


show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Stratified=ATEStratified("T","Y", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"])(values))


2025-07-21 15:43:11,627 - INFO - U : f(['N'])
2025-07-21 15:43:11,632 - INFO - T : f(['N', 'U'])
2025-07-21 15:43:11,635 - INFO - Y : f(['N', 'U', 'T'])
2025-07-21 15:43:11,639 - INFO - fitting model for propensity: 'propensity'
2025-07-21 15:43:11,762 - INFO - fitting model for imputation


,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute
0,0.522973,0.494279,0.49429,0.494272


## Information Leakage

In [8]:
SCM = {
    "U" : lambda N: (torch.rand(N) < 0.5)*1.0,
    "T" : lambda N, U: (torch.rand(N) < (0.1 + 0.4*U))*1.0,
    "Y" : lambda N, U, T: (torch.rand(N) < (0.1 + 0.4*U  + 0.5*T))*1.0,
    "Leakage" : lambda N, T, Y: (T+Y) * (torch.rand(N) < 0.3) 
}

values = eval_scm(SCM, 10000)
enrich_propensity(values, "T", ["U"], "propensity", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))
enrich_propensity(values, "T", ["U","Leakage"], "propensity_leakage", model=GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3))

mgen = lambda: GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

show(Naive=CondMeanDiff("T","Y")(values),
     ATE_Propensity=ATEPropensity("T","Y", "propensity")(values),
     ATE_Impute=ATEImpute("T","Y",["U"], mgen=mgen)(values),
     ATE_Propensity_Leakage=ATEPropensity("T","Y", "propensity_leakage")(values),
     ATE_Impute_Leakage=ATEImpute("T","Y",["U","Leakage"], mgen=mgen)(values))
     


2025-07-21 15:43:13,254 - INFO - U : f(['N'])
2025-07-21 15:43:13,256 - INFO - T : f(['N', 'U'])
2025-07-21 15:43:13,258 - INFO - Y : f(['N', 'U', 'T'])
2025-07-21 15:43:13,260 - INFO - Leakage : f(['N', 'T', 'Y'])
2025-07-21 15:43:13,262 - INFO - fitting model for propensity: 'propensity'
2025-07-21 15:43:13,644 - INFO - fitting model for propensity: 'propensity_leakage'
2025-07-21 15:43:14,140 - INFO - fitting model for imputation
2025-07-21 15:43:14,354 - INFO - fitting model for imputation


,Naive,ATE_Propensity,ATE_Impute,ATE_Propensity_Leakage,ATE_Impute_Leakage
0,0.683268,0.512974,0.512961,0.510864,0.438765


## Bidirectional Causality

Movie A can appear before or after Movie B

In [9]:

SCM = {
    "R1" : lambda N: torch.rand(N),
    "R2" : lambda N: torch.rand(N),
    "U" :  lambda N: (torch.rand(N) < 0.5)*1.0,
    "A1":  lambda N, U, R1: (R1 < (0.1 + 0.4*U))*1.0,
    "B" :  lambda N, U, A1, R2: (R2 < (0.1 + 0.4*U + 0.4*A1))*1.0,
    "A2":  lambda N, U, A1, B: torch.where(A1>0.5, torch.zeros(N), (torch.rand(N) < (0.1 + 0.4*U + 0.4*B))*1.0),
    "A" :  lambda A1, A2: (A1+A2 > 0.5) * 1.0
}

values = eval_scm(SCM, 100000)

ground_truth = ground_truth_ate(SCM, values, "A1","B")

enrich_propensity(values, "A1", ["U"])
enrich_propensity(values, "A", ["U"], "propensity_alt")


show(GroundTruth=ground_truth, 
     Naive=CondMeanDiff("A1","B")(values),
     ATE_Stratified=ATEStratified("A1","B", SplitStratify("U",[0.5]))(values),
     ATE_Propensity=ATEPropensity("A1","B", "propensity")(values),
     ATE_Impute=ATEImpute("A1","B",["U"])(values),
     ATE_Propensity_Alt=ATEPropensity("A1","B", "propensity_alt")(values),
     ATE_Propensity_Filtered=ATEPropensity("A","B", "propensity")(apply_filter(values, (values["A2"] < 0.5))),
     ATE_Impute_TisA=ATEImpute("A","B",["U"])(values),
    )



2025-07-21 15:43:14,701 - INFO - R1 : f(['N'])
2025-07-21 15:43:14,704 - INFO - R2 : f(['N'])
2025-07-21 15:43:14,707 - INFO - U : f(['N'])
2025-07-21 15:43:14,711 - INFO - A1 : f(['N', 'U', 'R1'])
2025-07-21 15:43:14,713 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-21 15:43:14,716 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-21 15:43:14,721 - INFO - A : f(['A1', 'A2'])
2025-07-21 15:43:14,723 - INFO - Calculating ground truth
2025-07-21 15:43:14,725 - INFO - DO A1
2025-07-21 15:43:14,726 - INFO - A1 : f(['N'])
2025-07-21 15:43:14,727 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-21 15:43:14,730 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-21 15:43:14,734 - INFO - A : f(['A1', 'A2'])
2025-07-21 15:43:14,736 - INFO - DO A1
2025-07-21 15:43:14,738 - INFO - A1 : f(['N'])
2025-07-21 15:43:14,740 - INFO - B : f(['N', 'U', 'A1', 'R2'])
2025-07-21 15:43:14,742 - INFO - A2 : f(['N', 'U', 'A1', 'B'])
2025-07-21 15:43:14,746 - INFO - A : f(['A1', 'A2'])
2025-07-21 15:43:14,749 - INFO - fi

,GroundTruth,Naive,ATE_Stratified,ATE_Propensity,ATE_Impute,ATE_Propensity_Alt,ATE_Propensity_Filtered,ATE_Impute_TisA
0,0.40049,0.585686,0.396605,0.396672,0.396601,0.331589,0.611309,0.495341


In [10]:
((values["A2"] < 0.5)*1.0).mean()
 


tensor(0.7629)